<a href="https://colab.research.google.com/github/hossain-mdismail/Cryptography_Lab_3/blob/main/Cryptography_Lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install cryptography

In [5]:
# All necessary imports for the entire lab
import sys
import os
import time
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

In [6]:
# Imports from our provided script
from AISC_03 import (
    encodeText, decodeText,
    generate_prime_candidate, is_prime, egcd,
    encryptAESCTR, decryptAESCTR,
    pDH, gDH, qDH
)

print("All libraries and functions imported successfully.")

All libraries and functions imported successfully.


#### PART 1: RSA WARM-UP (ENCRYPT FOR INSTRUCTOR)

In [22]:

# --- TODO:---

# 1. Getting the instructor's public key (N, e) from the Google Sheet

N_prof = 84679728665601623534964724907925515929006540043189517481604602443064696359792213876789134073333039587280256381629121330309212710075084136072434602842735932397920567074126295940793758270599694063151970957022891161811181118800418280560581409959172714364916927401331291518944188508877716559336163266657183044021

E_prof = 65537

# 2. Writing a short message to the instructor.
#    Keeping it short so it encodes to a single integer
MY_MESSAGE = "Hello Professor! This is group 07."


def main():
    print("Starting Lab 03 - Part 1: RSA Warm-up")

    # --- 1. Validation ---
    if N_prof is None or E_prof is None:
        print("\n" + "="*70)
        print(" ERROR: Please edit 'python file' and fill in")
        print("        'INSTRUCTOR_N' and 'INSTRUCTOR_E' with the values")
        print("        from the Google Sheet.")
        print("="*70)
        sys.exit(1)

    if "[Name]" in MY_MESSAGE:
        print("\n" + "="*70)
        print(" ERROR: Please edit 'lab03_part1.py' and change")

        print("="*70)
        sys.exit(1)

    print(f"Loaded Instructor's N (Modulus) with {N_prof.bit_length()} bits.")
    print(f"Loaded Instructor's E (Exponent): {E_prof}")
    print(f"Message to encrypt: '{MY_MESSAGE}'")

    # 2. Getting Bit Length
    # bitlen should be the length in bits of the RSA modulo N
    bitlen = N_prof.bit_length()

    # 3. Encoding the Message
    # Encode the string s into a list of integers [cite: 20, 21]
    m_list = encodeText(MY_MESSAGE, bitlen)

    # For the warm-up, we expect a single integer
    if len(m_list) > 1:
        print("\nERROR: Your message is too long and was encoded into")
        print(f"       {len(m_list)} integers. Please use a shorter message for this test.")
        sys.exit(1)

    m = m_list[0]
    print(f"\nEncoded message (m): {m}")

    # 4. Encrypting the Message
    # The ciphertext is c = m^e mod N
    # We use Python's pow(base, exp, mod) for this
    print("Encrypting... (this may take a moment)")
    ciphertext = pow(m, E_prof, N_prof)

    # 5. Posting the Result
    print("\n" + "="*70)
    print(" RSA Encryption Successful!")
    print("\nMy ciphertext is:")
    print(ciphertext)
    print("\n" + "="*70)
    print("COPYing this entire number and I have PASTE it into the Google Sheet")



if __name__ == '__main__':
    main()

Starting Lab 03 - Part 1: RSA Warm-up
Loaded Instructor's N (Modulus) with 1023 bits.
Loaded Instructor's E (Exponent): 65537
Message to encrypt: 'Hello Professor! This is group 07.'

Encoded message (m): 1369957976098687619078781968088907006431159387259052665595966240434733108964386120
Encrypting... (this may take a moment)

 RSA Encryption Successful!

My ciphertext is:
6083373696559605120837051464713499659002091358858452655348276561696725586758928433852828064860419772347429800526199706470573119936039723967286839645730826670759306379360350586248088669279946511851899194384668395279173231928328676840645859307227575126663455294813939117139839676032055595565878353082930046348

COPYing this entire number and I have PASTE it into the Google Sheet


#### PART 2: RSA KEY GENERATION (MY KEY PAIR)

In [23]:
# --- Configuration ---
KEY_BITS = 1024
PRIME_BITS = KEY_BITS // 2
PUBLIC_EXPONENT = 65537

print(f"Generating {KEY_BITS}-bit RSA key pair...")
p = 0
q = 0

# Generate prime 'p'
print(f"Finding first prime p ({PRIME_BITS} bits)...")
start_time = time.time()
while not is_prime(p, 100):
    p = generate_prime_candidate(PRIME_BITS)
print(f"Found p {p} \nin {time.time() - start_time:.2f} seconds.")

# Generate prime 'q'
print(f"Finding second prime q ({PRIME_BITS} bits)...")
start_time = time.time()
while not is_prime(q, 100) or p == q:
    q = generate_prime_candidate(PRIME_BITS)
print(f"Found q in {time.time() - start_time:.2f} seconds.")

# Calculate N and phi(N)
N = p * q
phiN = (p - 1) * (q - 1)

# Set public exponent 'e' and check
e = PUBLIC_EXPONENT
(g, x, y) = egcd(e, phiN)



Generating 1024-bit RSA key pair...
Finding first prime p (512 bits)...
Found p 11461683122892276693290154943016034115906080533870183860088623180588272541817959067145340546232821140642671398206338406505840653830312703850265898410871523 
in 0.11 seconds.
Finding second prime q (512 bits)...
Found q in 0.14 seconds.


In [24]:
if g != 1:
    print(f"ERROR: e={e} is not relatively prime to phiN. This is rare. Re-run cell.")
else:
    # Calculate private exponent 'd'
    d = x % phiN

    print("\n" + "="*70)
    print(" RSA KEY GENERATION COMPLETE")
    print("="*70)

    print("\n--- PUBLIC KEY (Post this on the Google Sheet) ---")
    print("\nMY_N (Modulus):")
    print(N)
    print("\nMY_E (Public Exponent):")
    print(e)

    print("\n\n--- PRIVATE KEY (need to keep this SECRET!) ---")
    print("\nMY_D (Private Exponent):")
    print(d)


 RSA KEY GENERATION COMPLETE

--- PUBLIC KEY (Post this on the Google Sheet) ---

MY_N (Modulus):
136520533950166462574781537723557585502503826170084906677643889124001732436951164365365488574282666257475359964979233001831603716688018068992500802826363139435535512594690045534034194036855119671947077134962158938548756517793029181885070907052871726622071076371795089082537922535122256860230141594943254768247

MY_E (Public Exponent):
65537


--- PRIVATE KEY (need to keep this SECRET!) ---

MY_D (Private Exponent):
119576547756861092988391055586086883449047517183291635229210860542671642691583497689321683028358757513738164518816895982485306745030170761773906702849409065891844009359613214491245188920675854621886084602706501699307196279796924315026503480502066545289063825280313679419390185245046139048839706874872454958057


In [25]:
print("\n\n--- PRIMES (For my report) ---")
print("\nMY_P:")
print(p)
print("\nMY_Q:")
print(q)
print("\n" + "="*70)



--- PRIMES (For my report) ---

MY_P:
11461683122892276693290154943016034115906080533870183860088623180588272541817959067145340546232821140642671398206338406505840653830312703850265898410871523

MY_Q:
11911037191169218724121433804854343058634870805644661301447917044622985096725215623492359435659147525978926327276484727298958381657304737449814515706384989



#### PART 3: RSA DECRYPTION (PROFESSOR'S REPLY)***

In [28]:

# --- Part 3 - RSA Decryption ---

print("Starting Part 3: RSA Decryption")

# --- TODO: EDIT THIS VALUE ---
# 1. Your key values from Cell 3 (I have filled these from your screenshot)
MY_N= 136520533950166462574781537723557585502503826170084906677643889124001732436951164365365488574282666257475359964979233001831603716688018068992500802826363139435535512594690045534034194036855119671947077134962158938548756517793029181885070907052871726622071076371795089082537922535122256860230141594943254768247
MY_D = 119576547756861092988391055586086883449047517183291635229210860542671642691583497689321683028358757513738164518816895982485306745030170761773906702849409065891844009359613214491245188920675854621886084602706501699307196279796924315026503480502066545289063825280313679419390185245046139048839706874872454958057



# 2. Get the instructor's reply (ciphertext) from the Google Sheet
#    You need to ask your professor to post this for you.
C_Prof = None   # <-- PASTE THE PROFESSOR'S 'c' REPLY HERE
# -------------------------------

if MY_N and MY_D and C_Prof:
    print("Decrypting instructor's reply...")

    # 1. Decrypt: m = c^d mod N
    decrypted_m = pow(C_Prof, MY_D, MY_N)

    # 2. Decode
    # decodeText expects a list of integers
    key_bitlen = MY_N.bit_length()
    decrypted_message = decodeText([decrypted_m], key_bitlen)

    print("\n" + "="*70)
    print(" ✅ Decryption Successful!")
    print("\nThe instructor's message is:")
    print(decrypted_message)
    print("="*70)
else:
    print("\n" + "❌ ERROR: Please paste the professor's reply")
    print("into the 'INSTRUCTOR_REPLY_C' variable and re-run this cell.")

Starting Part 3: RSA Decryption

❌ ERROR: Please paste the professor's reply
into the 'INSTRUCTOR_REPLY_C' variable and re-run this cell.


#### BONUS TASK (don't run this again, it will regenerate c,iv,ciphertext)

In [29]:
# --- Cell 7b: Bonus Task - RSA Key Encapsulation ---

print("Starting Bonus Task: RSA Key Encapsulation")

# --- TODO: EDIT THIS VALUE ---
# You need the INSTRUCTOR'S public key from Part 1
INSTRUCTOR_N = N_prof  # Paste instructor's N here
INSTRUCTOR_E = E_prof  # Paste instructor's E here (e.g., 65537)
# -------------------------------

# 1. Load the text file to encrypt
try:
    with open('text.txt', 'r', encoding='utf-8') as f:
        full_text_content = f.read()
    print(f"Loaded text.txt ({len(full_text_content)} characters)")
except FileNotFoundError:
    print("\n❌ ERROR: 'text.txt' not found. Please re-run Cell 1 and upload it.")
    full_text_content = None

if INSTRUCTOR_N and INSTRUCTOR_E and full_text_content:

    start_time = time.time()

    # 1. Generate a random 128-bit (16-byte) session key
    session_key_bytes = os.urandom(16)
    print(f"\nGenerated 16-byte AES session key.")

    time_key_gen = time.time() - start_time

    # 2. Encrypt the session key with the Instructor's RSA public key
    start_time = time.time()

    # Convert the 16-byte key to a single integer
    key_int = int.from_bytes(session_key_bytes, byteorder='little')

    # Encrypt the *integer* with RSA
    c_encapsulated_key = pow(key_int, INSTRUCTOR_E, INSTRUCTOR_N)

    time_key_encap = time.time() - start_time
    print(f"Encapsulated key with RSA.")

    # 3. Use the session key to encrypt a message with AES
    start_time = time.time()

    plaintext = full_text_content.encode('utf-8')
    # Use the *byte* key for AES
    (iv_bonus, ciphertext_bonus) = encryptAESCTR(session_key_bytes, plaintext)

    # Encode for Google Sheet
    iv_bonus_str = base64.b64encode(iv_bonus).decode('utf-8')
    ciphertext_bonus_str = base64.b64encode(ciphertext_bonus).decode('utf-8')

    time_aes_enc = time.time() - start_time
    print(f"Encrypted text.txt with AES.")

    # --- 4. Post these values to the Google Sheet ---
    print("\n" + "="*70)
    print(" ✅ BONUS TASK COMPLETE")
    print("\nPost these 3 values to the Google Sheet:")
    print("\nRSA encapsulated key (c):")
    print(c_encapsulated_key)

    print("\nAES iv (Base64):")
    print(iv_bonus_str)

    print("\nAES ciphertext (Base64):")
    print(ciphertext_bonus_str)
    print("="*70)

    # --- 5. Report these times ---
    print("\n 📊 PERFORMANCE (for your report)")
    print(f"Key Generation (16-byte): {time_key_gen:.6f} seconds")
    print(f"Key Encapsulation (RSA):  {time_key_encap:.6f} seconds")
    print(f"Message Encryption (AES): {time_aes_enc:.6f} seconds")
    print("="*70)

else:
    print("\n" + "❌ ERROR: Please fill in INSTRUCTOR_N and INSTRUCTOR_E")
    print("       and ensure 'text.txt' is uploaded.")

Starting Bonus Task: RSA Key Encapsulation
Loaded text.txt (10208 characters)

Generated 16-byte AES session key.
Encapsulated key with RSA.
Encrypted text.txt with AES.

 ✅ BONUS TASK COMPLETE

Post these 3 values to the Google Sheet:

RSA encapsulated key (c):
39920113603761353825295003968874642779490918908385128253268635710274863232762998968816042741772056470584229962811038812718246809796848767974044073892044731044689974519419601039441805289132056800296771593306465439372463748379098277172458624696648927879195576811826754499460335415214975346924301540446915740258

AES iv (Base64):
z88d9+FX202mjWckCvhGiQ==

AES ciphertext (Base64):
8iT0X+fF6q9K+Xfz6vCxUqRRoBZ5l4l7GoHDOriNpRTVj0w/wQQ4MJMURFCtKnyrImqo9J+I4IPvoMxsK3KyrdQX4CdDH/PxqDR4VF+t7OiB88d4AdupjgszXaJwysz/8hz98WThqNJgTn7vON5RJtmTdK4a1szpaeh0lC2SZtT02VdXfnEsGDuMG/NrZF+rnUjeIwoQ0Mw84BCqPVQOvm+KWXtCQHo6+OnaAu+fadZTEKeOojJJ6VaS/jOTxR7s2O5OGUuNPFF9Vjnh2FP4f4r3C9GHZIDCjgm4hk5pmZvxNWcJSF2uHeLdosAcbrETUFH6ZxQSb5S7COdaenpPawbI1NkQOjUELN0Tqof

#### Part 4a - Diffie-Hellman & AES Encryption

This cell generates our DH key, computes a shared AES key, and encrypts a message. we will Post our DH PUBLIC KEY to the sheet. Then, we will get another group's DH public key (which is 'pkDH' in the sheet) and paste it in.

In [21]:
# Part 4a - DH Key Exchange & AES Encryption

print("Starting Part 4: Diffie-Hellman and AES")

#Generating our DH key pair
print("Generating our DH key pair...")
# Creating a private key x in range [1, q-1]
dh_x = (int.from_bytes(os.urandom(256), byteorder='little') % (qDH - 1)) + 1 # this is my private key
#
# Computing public key pk = g^x mod p
my_pkDH = pow(gDH, dh_x, pDH)   # we can share this public key

print("\n--- OUR DH PUBLIC KEY (Posted this on the Google Sheet) ")
print("\nMY_DH_PK:")
print(my_pkDH)
print("\n(Keeping our 'dh_x' (private key) value SECRET!)")


# Getting professor's DH public key from the sheet
PK_prof = 231496621204370508895931748857755688773955557370722566546524279241681783924615459222281328781846958709671178933550268979137824811356380142723458582689238338881602937866599482596517635627201490862987705580528580122030574292913825460656414418530301010266004326589419172467364887958500892750202983947301330827393531207626744229962720413918512823808690023548441036635272327523845223428985955360366428904878223170012768633135103661638106507748637163377957664799771569021118234716556142527020139132381453117289393367071863842364636961374713484929024198420198291374856545022707013121483896955505081060905509151197548949686

#  Message to send via AES
AES_MESSAGE = "This is a secret message encrypted with AES-CTR!"
# -------------------------------

if PK_prof:
    # Computing shared key k = (pk_2)^x mod p
    print("\nComputing shared secret...")
    k = pow(PK_prof, dh_x, pDH)

    # Deriving 128-bit (16-byte) AES key from k

    shared_key_aes = (k & ((1 << 128) - 1)).to_bytes(16, byteorder='little')
    print(f"Derived 16-byte AES key.")

    # Encrypting with AES-CTR
    print(f"Encrypting message: '{AES_MESSAGE}'")
    plaintext = AES_MESSAGE.encode('utf-8')
    (iv, ciphertext) = encryptAESCTR(shared_key_aes, plaintext)

    # Encoding for Google Sheet using Base64
    iv_string = base64.b64encode(iv).decode('utf-8')

    ciphertext_string = base64.b64encode(ciphertext).decode('utf-8')

    print("\n" + "="*70)
    print(" AES Encryption Successful!")
    print("\nneed to Post these two (dynamic) strings on the Google Sheet:")
    print(f"\nIV (Base64):\n{iv_string}")
    print(f"\nCiphertext (Base64):\n{ciphertext_string}")
    print("="*70)
else:
    print("\n" + "❌ ERROR: Please fill in OTHER_GROUP_PK")
    print("in the cell above before running this one.")

Starting Part 4: Diffie-Hellman and AES
Generating our DH key pair...

--- OUR DH PUBLIC KEY (Posted this on the Google Sheet) 

MY_DH_PK:
390770603400099216996218930192697211980988614174469462096235543726967708080382935686224429813782439240564948653613237520415598046705790958758002335399389020590030342742047772821198577154043710751632887230228240872479140297114314921405328133376725976922199846299545408034391188532552268973034237376384318555318430446630502245965270671860442274513415654028709511466740212689758042002886544956915072239258109436397310245849471157813635035094147626468096569191606643097141841434140194693777034633424646513249206629409696909005888131588188634328413662248677738892132558259415423383462358309268753350316446313209804493807

(Keeping our 'dh_x' (private key) value SECRET!)

Computing shared secret...
Derived 16-byte AES key.
Encrypting message: 'This is a secret message encrypted with AES-CTR!'

 AES Encryption Successful!

need to Post these two (dynamic) strings 

#### Part- 4b: AES Decryption: ***

In [30]:

print("Starting Part 4: AES Decryption")

# --- TODO: EDIT THESE VALUES ---
# 1. Get the values from the *professor* on the Google Sheet
#    (The ones under "AES encrypted messages with DH key (from professor)")
IV_string_prof = None         # <-- PASTE THE PROFESSOR'S IV STRING HERE
C_string_prof = None # <-- PASTE THE PROFESSOR'S CIPHERTEXT STRING HERE

# 2. 'shared_key_aes' must exist from running Cell 5.
#    Do NOT re-run Cell 5, or your key will change and won't match!
# -------------------------------

if (IV_string_prof and C_string_prof and
    'shared_key_aes' in locals() and shared_key_aes is not None):

    print("Decoding and decrypting professor's message...")

    # 1. Decode from base64
    try:
        iv_from_sheet = base64.b64decode(IV_string_prof)
        ciphertext_from_sheet = base64.b64decode(C_string_prof)

        # 2. Decrypt
        # This uses the 'shared_key_aes' that is still in memory from Cell 5
        decrypted_payload_bytes = decryptAESCTR(shared_key_aes,
                                                iv_from_sheet,
                                                ciphertext_from_sheet)

        # 3. Convert from bytes to a readable string
        decrypted_payload_string = decrypted_payload_bytes.decode('utf-8')

        print("\n" + "="*70)
        print(" ✅ AES Decryption Successful!")
        print("\nThe professor's secret message is:")
        print(decrypted_payload_string)
        print("="*70)

    except Exception as e:
        print(f"\n❌ ERROR: Decryption failed. Are the strings correct? {e}")
        print("       (This can also happen if the professor used a different key)")
else:
    print("\n"V" + "❌ ERROR: Please fill in IV_string_prof and C_string_prof.")
    print("       You must also run Cell 5 *once* (and not again) to generate")
    print("       the 'shared_key_aes' before running this cell.")

SyntaxError: invalid character '❌' (U+274C) (ipython-input-2416500871.py, line 42)